安装包

In [ ]:

%pip install -r requirements.txt
%pip install "deepspeed==0.14.0" --upgrade
%pip install modelscope
%pip install flash-attn --no-build-isolation

!pip install accelerate==0.26.1
!pip install bitsandbytes==0.42.0
!pip install ctranslate2==3.24.0
!pip install datasets==2.16.1
!pip install peft==0.7.1
!pip install trl==0.7.10
!pip install typing-extensions==4.7.1
!pip install transformers==4.41.1
!pip install sentencepiece
!pip install sacrebleu sentencepiece -q
!pip install flash-attn --no-build-isolation
!pip install pandas
!pip install cdifflib
!pip install openpyxl

In [ ]:
dbutils.library.restartPython()

LoRA训练

In [ ]:
import os
from pyspark.ml.torch.distributor import TorchDistributor
import mlflow
import torch
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["NCCL_IGNORE_DISABLED_P2P"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["TORCH_EXTENSIONS_DIR"] = "/root/.cache/torch_extensions/py310_cu118"
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ["DATABRICKS_HOST"] = host
os.environ["DATABRICKS_TOKEN"] = mlflow.utils.databricks_utils.get_databricks_host_creds().token
username = spark.sql("SELECT current_user()").first()['current_user()']
experiment_path = f'/Users/{username}/finetune-aya-8b'
mlflow.set_experiment(experiment_path)
os.environ["EXPERIMENT_PATH"] = experiment_path
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

# 设置参数
base_path = "/Volumes/main/default/default_volume/erikyzzhang/HOK"
model_name_or_path = base_path + "/models/models--CohereForAI--aya-23-8B"
model_max_length = 256
output_dir = base_path + "/models/"
data_path = '/Volumes/main/default/default_volume/erikyzzhang/HOK/MOBA-HOK-train.json'
use_databricks = True

# 如果输出目录不存在，创建它
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

NUM_PROCESSES = torch.cuda.device_count()
print(f"We're using {NUM_PROCESSES} GPUs")
single_node_multi_gpu_ckpt_path = TorchDistributor(num_processes=NUM_PROCESSES, local_mode=True, use_gpu=True) \
    .run("./supervised-fine-tune-hok.py", f'--model_name_or_path={model_name_or_path}', f"--data_path={data_path}",
    f'--model_max_length={str(model_max_length)}', 
    f'--output_dir={output_dir}',
    f'--use_databricks', use_databricks)

In [ ]:
!pip install sacrebleu sentencepiece -q
dbutils.library.restartPython()

In [ ]:
from peft import PeftModel, PeftConfig
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
import torch
cache_dir = "./"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,)

#/Volumes/main/default/default_volume/erikyzzhang/aya_pubg_ft_/checkpoint-59/
# output_directory = "/Volumes/main/default/default_volume/erikyzzhang/aya_hok_ft_"
output_directory = "/Volumes/main/default/default_volume/erikyzzhang/HOK/models"

cache_dir = "./"
model_name = "CohereForAI/aya-23-8B"
peft_model_path = os.path.join(output_directory, "checkpoint-382")  # change checkpoint path

peftconfig = PeftConfig.from_pretrained(peft_model_path)

model_base = AutoModelForCausalLM.from_pretrained(peftconfig.base_model_name_or_path, quantization_config=nf4_config, 
                                             device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token = True, add_eos_token = False, force_download=True,)

new_model = PeftModel.from_pretrained(model_base, peft_model_path)

print("Peft model loaded")

In [ ]:
import json
import os
from sentence_transformers import SentenceTransformer
from transformers import GenerationConfig
import torch
import csv

def generate_response(prompt, model):
  encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to("cuda")
  with torch.no_grad():
    generated_ids = model.generate(**model_inputs,
                                  max_new_tokens=128,
                                  min_new_tokens=1,
                                  do_sample=False,
                                  pad_token_id=tokenizer.eos_token_id)
        
  decoded_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  return decoded_output[0].replace(prompt, "")

def load_json(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return None

translations = []
references = []

test_file = '/Volumes/main/default/default_volume/erikyzzhang/MOBA-HOK-cn-en-test.json'
with open(test_file, 'r', encoding='utf-8') as file:
    translation_pairs = json.load(file)
print(len(translation_pairs))
for pair in translation_pairs:
    source_lang = pair['source_language']
    source_text = pair['source_text']
    target_lang = pair['target_language']
    target_text = pair['target_text']
    if source_text != 'nan' and target_text != 'nan':
        example = "Translate the  Text from " + source_lang +  " to " + target_lang + "." + "\n" +  source_lang + ": " + source_text    
        prompt = f"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{example}<|END_OF_TURN_TOKEN|><|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>"
        references.append(target_text)
        response = generate_response(prompt, new_model)
        start_index = response.find("<|CHATBOT_TOKEN|>")
        response = response[start_index:].replace("<|CHATBOT_TOKEN|>", "")
        print(response)
        translations.append(response)    
print(len(translations))



In [ ]:
print(references[-5:])
print(translations[-5:])
print(len(references))

In [ ]:
import sacrebleu
# Calculate BLEU
bleu = sacrebleu.corpus_bleu(translations, [references])  # for spBLEU: tokenize='flores200'
bleu = round(bleu.score, 2)
print("BLEU:", bleu)

# Calculate chrF++
chrf = sacrebleu.corpus_chrf(translations, [references], word_order=2)  # for chrF++ word_order=2
chrf = round(chrf.score, 2)
print("chrF++:", chrf)

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(translations, [references])
ter = round(ter.score, 2)
print("TER:", ter)